# Bottom Temperature Time Series for Pan005

In [1]:
import gsw
import glob
import dask.distributed as dsk
import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr
import numpy as np
import xesmf as xe

import cmocean as cm
import cartopy.crs as ccrs

import cartopy.feature as cft
import matplotlib.path as mpath

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

In [2]:
import logging
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [3]:
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34861,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:46713,Total threads: 1
Dashboard: /proxy/42955/status,Memory: 4.47 GiB
Nanny: tcp://127.0.0.1:33923,


2025-07-02 10:12:15,079 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:33211 (pid=1498379) exceeded 95% memory budget. Restarting...
2025-07-02 10:12:15,123 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33211' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'original-open_dataset-thetao-0b9923213d6d38e8c08816bd01c4bfe1', ('getitem-d3aaafdb96eed64c3e535b6b7330129c', 99, 0, 0, 0), ('shuffle-split-cea311f8414f002b77097729e222f94d', 39), ('concatenate-getitem-mean_chunk-30276130360cf0e264e20ed78129449d', 98, 0, 1, 5), ('open_dataset-thetao-ddb787228e0f356898a681aabd906dde', 0, 0, 2, 0), ('open_dataset-thetao-ddb787228e0f356898a681aabd906dde', 0, 0, 2, 4), 'original-open_dataset-thetao-e49df6c31176c184844b94e5c10b5214', ('getitem-d3aaafdb96eed64c3e535b6b7330129c', 92, 0, 0, 0), ('shuffle-split-0bce543fcb0678d2436fde02b9307a27', 39)} (stimulus_id='handle-worker-cleanup-1751415135.1233666')
2025-07-02 10:12:15,222

In [4]:
pathsC = np.sort(glob.glob('/g/data/ol01/outputs/mom6-panan/panant-005-zstar-ACCESSyr2/outpu*/*.ocean_month_z.nc'))
pathsM = np.sort(glob.glob('/g/data/g40/akm157/model_output/mom6-panan/panant-005-zstar-ssp126-MW-only/outpu*/*.ocean_month_z.nc'))[:60]

In [5]:
shelf_mask = shelf_mask = xr.open_dataset('/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_005deg.nc')['contour_masked_above']

In [6]:
#Time variant variables
def preprocess(ds):
    ds = ds[['thetao', 'volcello','so']].sel(yh=shelf_mask['yh']).sel(xh=shelf_mask['xh'])
    return ds

cont = xr.open_mfdataset(pathsC, preprocess = preprocess, chunks = 'auto')
melt = xr.open_mfdataset(pathsM, preprocess = preprocess, chunks = 'auto')

In [7]:
#Time slicing
cont = cont.sel(time=slice('2001-01-01','2010-12-31'))
melt = melt.sel(time=slice('2001-01-01','2010-12-31'))

In [8]:
#Time invariant variables
depth = xr.open_dataset('/g/data/ol01/outputs/mom6-panan/panant-005-zstar-ACCESSyr2/output239/20101201.ocean_static.nc')['deptho']
areaC = xr.open_dataset('/g/data/ol01/outputs/mom6-panan/panant-005-zstar-ACCESSyr2/output239/20101201.ocean_static.nc')['areacello'].sel(xh=shelf_mask['xh'])
areaM = xr.open_dataset('/g/data/g40/akm157/model_output/mom6-panan/panant-005-zstar-ssp126-MW-only/output120/20010101.ocean_static.nc')['areacello'].sel(xh=shelf_mask['xh'])

In [9]:
# Spatial mean
cont_time = cont.mean(dim=['xh','yh'])
melt_time = melt.mean(dim=['xh','yh'])

In [11]:
# Calculating pressure
p_cont = gsw.p_from_z(-cont_time['z_l'], cont['yh'])
p_melt = gsw.p_from_z(-melt_time['z_l'], melt['yh'])

# Calculating absolute salinity
SA_cont = gsw.SA_from_SP(cont_time['so'], p_cont, cont['xh'], cont['yh'])
SA_melt =  gsw.SA_from_SP(melt_time['so'], p_melt, melt['xh'], melt['yh'])

# Taking mean
SA_cont = SA_cont.mean(dim=['xh','yh'])
SA_melt = SA_melt.mean(dim=['xh','yh'])

# Calculating conservative temperature
tempC = gsw.CT_from_pt(SA_cont,cont_time['thetao'])
tempM = gsw.CT_from_pt(SA_melt,melt_time['thetao'])

In [12]:
tempC

<xarray.DataArray 'so' (time: 120, z_l: 75)> Size: 72kB
dask.array<ct_from_pt, shape=(120, 75), dtype=float64, chunksize=(1, 75), chunktype=numpy.ndarray>
Coordinates:
  * z_l      (z_l) float64 600B 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time     (time) object 960B 2001-01-16 12:00:00 ... 2010-12-16 12:00:00

In [ ]:
tempC = tempC.astype('float32').load()

2025-07-02 10:11:52,201 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.16 GiB -- Worker memory limit: 4.47 GiB
2025-07-02 10:11:52,478 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.72 GiB -- Worker memory limit: 4.47 GiB
2025-07-02 10:12:05,017 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 3.58 GiB -- Worker memory limit: 4.47 GiB
2025-07-02 10:12:05,060 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memo

In [ ]:
tempM = tempM.astype('float32').load()

In [ ]:
# To fix the label issues
tempC = xr.DataArray(tempC, dims=cont_time['thetao'].dims, coords=cont_time['thetao'].coords)
tempC.attrs['units'] = '°C'
tempC.name = 'CT'

tempM = xr.DataArray(tempM, dims=melt_time['thetao'].dims, coords=melt_time['thetao'].coords)
tempM.attrs['units'] = '°C'
tempM.name = 'CT'

In [ ]:
bottom_mask_tempC = tempC.where(~np.isfinite(tempC.shift({'z_l': -1})))
bottom_mask_tempC = ~np.isnan(bottom_mask_tempC)

bottom_mask_tempM = tempM.where(~np.isfinite(tempM.shift({'z_l': -1})))
bottom_mask_tempM = ~np.isnan(bottom_mask_tempM)

In [ ]:
bottemp_cont = tempC.where(bottom_mask_tempC).sum(dim='z_l')
bottemp_pert = tempM.where(bottom_mask_tempM).sum(dim='z_l')

In [ ]:
bottemp_pert

In [ ]:
bottemp_anom = bottemp_pert - bottemp_cont
bottemp_anom = bottemp_anom.load()

In [ ]:
bottemp_anom.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/pan005_bottemp_CT_anom.nc')

# Intermodel comparison

In [ ]:
pan005_bottemp_anom = xr.open_dataarray('/g/data/g40/kc5856/access_panan_ssp126_SWMT/pan005_bottemp_CT_anom.nc', chunks = 'auto')
pan01_bottemp_anom = xr.open_dataarray('/g/data/g40/kc5856/access_panan_ssp126_SWMT/pan01_bottemp_CT_anom.nc', chunks = 'auto')
acc_bottemp_anom = xr.open_dataarray('/g/data/g40/kc5856/access_panan_ssp126_SWMT/access_bottemp_anom.nc', chunks = 'auto')

In [ ]:
pan005_bottemp_anom_new = pan005_bottemp_anom.to_series()
pan01_bottemp_anom_new = pan01_bottemp_anom.to_series()
acc_bottemp_anom_new = acc_bottemp_anom.to_series()

In [ ]:
import pandas as pd

In [ ]:
# Slice 1910–1919 and 2001–2010
pan005_bottemp_anom_new = pan005_bottemp_anom_new['2001-01-01':'2010-10-16']
pan01_bottemp_anom_new = pan01_bottemp_anom_new['2001-01-01':'2010-10-16']
acc_bottemp_anom_new = acc_bottemp_anom_new['1910-01-01':'1919-10-16']

# Reset index to normalized months
pan005_bottemp_anom_new.index = range(1, len(pan005_bottemp_anom_new)+1)
pan01_bottemp_anom_new.index = range(1, len(pan01_bottemp_anom_new)+1)
acc_bottemp_anom_new.index = range(1, len(acc_bottemp_anom_new)+1)

In [ ]:
# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(pan005_bottemp_anom_new.index, pan005_bottemp_anom_new.values, label='PanAntarctic 1/20th')
plt.plot(pan01_bottemp_anom_new.index, pan01_bottemp_anom_new.values, label='PanAntarctic 1/10th')
plt.plot(acc_bottemp_anom_new.index, acc_bottemp_anom_new.values, label='ACCESS-OM2-01')

# Set x-axis ticks to represent years 1 through 10
plt.xticks(ticks=range(1, 121, 12), labels=[str(i) for i in range(1, 11)])

# Labeling and formatting
plt.xlabel('Time (years)')
plt.ylabel("Bottom temperature anomaly ($^\circ$C)")
plt.title('Bottom temperature anomaly of Panan01 vs ACCESS-OM2-01, shelf only')
plt.legend()
plt.grid(True, color='lightgrey')
plt.tight_layout()
plt.show()